### --- Day 25: Full of Hot Air ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/25

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2022, 25);
WriteLines(inputLines);

Loading puzzle file: Day25.txt
Total lines: 107
Max line length: 20

1-=022001=-1-
2-1020=0-01=2000
1==-2=122-=2-21102
1-0=1=-0-==0=--=-2
1-0-1=--22=


In [4]:
string[] testInputLines = 
[
    "1=-0-2",
    "12111",
    "2=0=",
    "21",
    "2=01",
    "111",
    "20012",
    "112",
    "1=-1=",
    "1-12",
    "12",
    "1=",
    "122",
];

Ok, I think this one looks reasonably straightforward. Converting from SNAFU to int should be a matter of converting, multiplying, and adding the characters, but converting to SNAFU looks a bit trickier, so we'll start with that one.

The SNAFU number is essentially base 5, except that once we get to the numbers 3, 4 we need to "carry over": eg:

* `20` in base 10 equals `40` in base 5.
* `40` in base 5 equals `[1, -1, 0]` in base SNAFU. That is $1 * 25$, $-1 * 5$, and $0$, which sums to 20.
* `[1, -1, 0]` in base SNAFU converts to the chars `1-0`.

Let's do this in the separate stages: convert to base 5, carry over, then convert to SNAFU chars.

In [5]:
IList<int> IntToBase5(long input)
{
    List<int> result = new();

    long remain = input;
    while (remain > 0)
    {
        int baseVal = (int)(remain % 5);
        result.Insert(0, baseVal);
        remain = remain / 5;
    }

    return result;
}

IntToBase5(20).Display();

[ 4, 0 ]

In [6]:
IList<int> Base5ToSnafu(IList<int> base5)
{
    var result = base5.ToList();

    result.Insert(0, 0); // in case the first digit carries over

    foreach (var i in Range(0, result.Count).Reverse())
    {
        var iVal = result[i];
        if (iVal > 2)
        {
            // Carry over
            result[i-1] += 1;
            result[i] = (iVal - 5);
        }
    }

    if (result[0] is 0)
    {
        // First digit didn't carry over
        result.RemoveAt(0);
    }
    
    return result;
}

Base5ToSnafu(IntToBase5(20)).Display();

[ 1, -1, 0 ]

In [7]:
string SnafuToString(IList<int> snafu) => new(snafu.Select(d => d switch {
    -1 => '-',
    -2 => '=',
    _ => d.ToString()[0]
}).ToArray());

string IntToSnafu(long base10) => SnafuToString(Base5ToSnafu(IntToBase5(base10)));

Console.WriteLine(IntToSnafu(20));

1-0


That takes care of the conversion to SNAFU. Now let's convert back to integer.

In [8]:
long SnafuToInt(string snafu)
{
    long @base = 1;
    long result = 0;
    foreach (var ch in snafu.Reverse())
    {
        var num = ch switch {
            '-' => -1,
            '=' => -2,
            _ => int.Parse($"{ch}")
        };

        result += (num * @base);
        @base *= 5;
    }

    return result;
}

Console.WriteLine(SnafuToInt("1-0"));

20


Putting it all together....

In [9]:
string SumAndSnafu(string[] inputLines) =>
    IntToSnafu(inputLines.Select(SnafuToInt).Sum());


In [10]:
// Reversing the process, you can determine that for the decimal number 4890,
// the SNAFU number you need to supply to Bob's console is 2=-1=0.

var testAnswer = SumAndSnafu(testInputLines);
Console.WriteLine(testAnswer);

2=-1=0


In [11]:
// The Elves are starting to get cold. What SNAFU number do you supply to Bob's console?

var part1Answer = SumAndSnafu(inputLines);
Console.WriteLine(part1Answer);

2-0-01==0-1=2212=100


In [12]:
// 2-0-01==0-1=2212=100 is correct!
Ensure("2-0-01==0-1=2212=100", part1Answer);

And that's it for another year! 🥳 Many thanks to Eric Wastl and everyone involved with Advent of Code for donating their time and energy to create these puzzles. I would like to say "see you next year", but since I'm working backwards through the catalogue, I guess I'll see you, er, last year! ;)